<a href="https://colab.research.google.com/github/SDW1014/python_albamon_coloring_to_telegram_channel/blob/main/Channel_Auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output

# 8080 포트를 사용하여 자동 재연결 설정
output.serve_kernel_port_as_iframe(port=8080)

import telebot
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

def split_message(message):
    message_chunks = []
    while len(message) > 0:
        if len(message) <= 4000:
            message_chunks.append(message)
            break
        else:
            chunk = message[:4000]
            last_space = chunk.rfind(' ')
            if last_space == -1:
                last_space = 4000
            message_chunks.append(chunk[:last_space])
            message = message[last_space+1:]
    return message_chunks



# 검색 결과를 저장할 리스트
search_results = []

# 봇의 토큰을 입력해주세요.
bot = telebot.TeleBot('')

@bot.message_handler(commands=['search_alba'])
def search_alba(message):
    bot.reply_to(message, "검색 시작")
    session = requests.Session()

    conn = sqlite3.connect('job_postings.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS job_postings (title TEXT, company TEXT, url TEXT)')

    c.execute('SELECT title, company, url FROM job_postings')
    job_postings = c.fetchall()

    url = "https://www.albamon.com/jobs/telecommuting?page={}"
    page = 1

    print("데이터 베이스 여기까지임")

    while True:
        response = session.get(url.format(page))
        soup = BeautifulSoup(response.content, 'lxml')

        jobs = soup.find_all('div', class_='list-item-recruit__contents--article')

        if not jobs:
            print("No more pages available")
            break

        new_postings = []
        for job in jobs:
            title_element = job.find('span', class_='typography-paid')
            company_element = job.find('span', class_='list-item-recruit__company-name')
            job_url = job.find('a', class_='list-item-recruit__link')['href']
            title = title_element.text.strip() if title_element else ''
            company = company_element.text.strip() if company_element else '광고'
            posting = (title, company, job_url)

            if posting not in job_postings:
                new_postings.append(posting)

        if new_postings:
            new_postings.reverse()
            c.executemany('INSERT INTO job_postings (title, company, url) VALUES (?, ?, ?)', new_postings)
            conn.commit()

            job_postings = new_postings + job_postings

        time.sleep(1)
        page += 1
    bot.reply_to(message, "검색 완료")

@bot.message_handler(commands=['select_alba'])
def select_alba(message):
    

    bot.reply_to(message, "검색어를 입력해주세요.")

    @bot.message_handler(func=lambda message: True)
    def get_search_keywords(message):
        search_keywords = message.text.strip()

        bot.reply_to(message, "제외어를 입력해주세요.")

        @bot.message_handler(func=lambda message: True)
        def get_exclude_keywords(message):
            exclude_keywords = message.text.strip()
            conn = sqlite3.connect('job_postings.db')
            c = conn.cursor()
            # Convert the input strings into lists of individual keywords
            search_list = [keyword.strip() for keyword in search_keywords.split(',')]
            exclude_list = [keyword.strip() for keyword in exclude_keywords.split(',')]

            # Build the SQL query using the search and exclude keywords
            query = "SELECT * FROM job_postings WHERE "
            for i, keyword in enumerate(search_list):
                if i > 0:
                    query += "AND "
                query += f"title LIKE '%{keyword}%' "
            for keyword in exclude_list:
                query += f"AND title NOT LIKE '%{keyword}%' "

            # Search the job_postings table for the input keywords
            c.execute(query)
            results = c.fetchall()

            if len(results) == 0:
                bot.reply_to(message, '검색 결과가 없습니다.')
            else:
                bot.reply_to(message, f'"{search_keywords}"에 대한 검색 결과입니다.\n')
                for result in results:
                    # Create message for each search result
                    result_message = f'Title: {result[0]}\nCompany: {result[1]}\nURL: {result[2]}\n\n'
                    # Split message into chunks of 4000 characters
                    result_chunks = split_message(result_message)
                    # Send each chunk as a separate message
                    for chunk in result_chunks:
                        bot.send_message(message.chat.id, chunk)
                        # Wait for 1 second before sending the next message
                        time.sleep(0.3)              
                #bot.reply_to(message, f'"{search_keywords}"에 대한 검색 결과입니다.\n')
                #for result in results:
                    # Create message for each search result
                    #result_message = f'Title: {result[0]}\nCompany: {result[1]}\nURL: {result[2]}\n\n'
                    # Split message into chunks of 4000 characters
                    #result_chunks = split_message(result_message)
                    # Send each chunk as a separate message
                    #for chunk in result_chunks:
                        #bot.send_message(message.chat.id, chunk)

            # Close the database connection
            conn.close()

        bot.register_next_step_handler(message, get_exclude_keywords)

    bot.register_next_step_handler(message, get_search_keywords)
bot.polling()

<IPython.core.display.Javascript object>

In [ ]:
pip install telebot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.10.0-py3-none-any.whl size=205421 sha256=d5a359baefc1d32b064119514257a732185de416fa214278596ce4c315a84bbf
  Stored in directory: /root/.cache/pip/wheels/ef/ba/4b/187be68f61aa7818b28d8b21200d130fd35ba36f002efcc262
Successfully built pyTelegramBotAPI


In [ ]:
pip uninstall telegram

In [ ]:
!pip install python-telegram-bot

In [ ]:
!pip install python-telegram-bot --upgrade